In [1]:
import pandas as pd
import sys
import os
import numpy as np

current_dir = os.path.abspath('')
parent_dir = os.path.join(current_dir, '..', 'functions')
sys.path.append(parent_dir)

from duplicates_utils import normalize_id
from mn_table_utils import create_mn_table
from one_to_one_table_utils import create_one_to_one_table
from duplicates_utils import number_of_duplicates

DROP DUPLICATES ROWS FROM AUTHORS AND CATEGORIES

In [2]:
authors_path = "../../dataset/cleaned/authors_cleaned.csv"
df_authors = pd.read_csv(authors_path)

print(df_authors.shape)

df_authors,df_authors_map = normalize_id(df_authors, key_column = 'author_name', id_column = 'author_id')

print(df_authors.shape)
print(len(df_authors_map))

#Reduce the dictionary only to items where the key and value are different
df_authors_map = {k: v for k, v in df_authors_map.items() if k != v}


(654021, 2)
(653783, 2)
654021


In [3]:
categories_path = "../../dataset/cleaned/categories_cleaned.csv"
df_categories = pd.read_csv(categories_path)


print(df_categories.shape)

df_categories,df_categories_map = normalize_id(df_categories, key_column = 'category_name', id_column = 'category_id')

print(df_categories.shape)
print(len(df_categories_map))

#Reduce the dictionary only to items where the key and value are different
df_categories_map = {k: v for k, v in df_categories_map.items() if k != v}

(2775, 2)
(2585, 2)
2775


CREATE M:N TABLES FROM MAIN DATASET

In [4]:
dataset_path = "../../dataset/cleaned/dataset_cleaned.csv"
df_dataset = pd.read_csv(dataset_path)


In [5]:
#AUTHOR MN TABLE
authors_mn_table = create_mn_table(df_dataset, id_column = 'id', key_column = 'authors')

#adds unknown id instead of missing id in mn table
unknown_author_id = df_authors.loc[df_authors['author_name'] == 'Unknown', 'author_id'].iloc[0]

#removes 'Unknown' author from df_authors dataset
df_authors = df_authors.drop((df_authors[df_authors["author_id"] 
                                         == unknown_author_id].index))


authors_mn_table['authors_id'] = authors_mn_table['authors_id'].str.strip()
authors_mn_table['authors_id'] = authors_mn_table['authors_id'].replace('', np.nan)
authors_mn_table['authors_id'] = authors_mn_table['authors_id'].astype('Int64')

authors_mn_table['authors_id'] = (authors_mn_table['authors_id'] 
    .replace(df_authors_map))

#remove unknown_author_id author id
authors_mn_table["authors_id"] = (authors_mn_table["authors_id"]
                                  .replace(unknown_author_id, np.nan))

authors_mn_table = authors_mn_table.dropna(subset=["authors_id"])

#drop duplicates
authors_mn_table = authors_mn_table.drop_duplicates()



In [6]:
#CATHEGORIES MN TABLE
categories_mn_table = create_mn_table(df_dataset, id_column = 'id', key_column = 'categories')

categories_mn_table['categories_id'] = categories_mn_table['categories_id'].str.strip()
categories_mn_table['categories_id'] = categories_mn_table['categories_id'].replace('', np.nan)
categories_mn_table['categories_id'] = categories_mn_table['categories_id'].astype('Int64')

categories_mn_table['categories_id'] = (categories_mn_table['categories_id'] 
    .replace(df_categories_map))

#remove empty rows
categories_mn_table = categories_mn_table.dropna(subset=["categories_id"])

#remove duplicates
categories_mn_table = categories_mn_table.drop_duplicates()

CREATE 1:1 TABLE FROM MAIN DATASET

In [7]:
lang_dataset, lang_mapping = create_one_to_one_table(df_dataset,'lang')

df_dataset['lang'] = (df_dataset['lang']
    .replace('', np.nan)  
    .map(lang_mapping)) 

In [8]:
lang_dataset

,id,lang
0,1,en
1,2,es
2,3,nan
3,4,it
4,5,de
...,...,...
158,159,art
159,160,tig
160,161,tai
161,162,lg


DROP OLD COLUMNS

In [9]:
df_dataset = df_dataset.drop(columns=['authors','categories'])

CONVERT TO THE CORRECT COLUMN TYPES

In [10]:
df_dataset[["format", "lang"]] = df_dataset[
                                ["format", "lang"]].astype("Int64")



RENAME COLUMNS

In [11]:
authors_mn_table

,main_id,authors_id
0,1,1
1,2,2
2,2,3
3,3,4
4,4,5
...,...,...
1615318,1109380,336369
1615319,1109381,29792
1615320,1109381,654019
1615321,1109382,654020


In [12]:
df_dataset = df_dataset.rename(columns=
                               {'description': 'book_description',
                                 'format': 'format_id',
                                   'lang': 'language_id',
                                   'id': 'book_id',
                                   'publication-date': 'publication_date'})

categories_mn_table = categories_mn_table.rename(columns=
                                          {'main_id': 'book_id',
                                           'categories_id':'category_id'})

authors_mn_table = authors_mn_table.rename(columns=
                                          {'main_id': 'book_id',
                                            'authors_id':'author_id'})

lang_dataset = lang_dataset.rename(columns=
                                          {'id': 'language_id',
                                            'lang':'language_name'})

In [13]:
new_order_columns = ['book_id','title','book_description','isbn13',
                     'publication_date', 'format_id', 'language_id']


df_dataset = df_dataset[new_order_columns]

SAVE FILES

In [14]:

def save_dataframes(df_dict, folder_path, file_format='csv'):
    if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
        raise FileNotFoundError(f"Dictionary '{folder_path}' does not exist.")

    for file_name, df in df_dict.items():
        file_name = f"{file_name}.{file_format}" 
        file_path = os.path.join(folder_path, file_name)  

        if file_format == 'csv':
            df.to_csv(file_path, index=False, encoding='utf-8')
        elif file_format == 'xlsx':
            df.to_excel(file_path, index=False, engine='openpyxl')
        else:
            raise ValueError("Unsuported file format.")

        print(f"✅ File was save: {file_path}")



In [15]:
dfs = {'final_dataset': df_dataset, 'final_categories': df_categories,
       'final_authors': df_authors,'mn_author': authors_mn_table,
       'mn_categories': categories_mn_table,'final_lang': lang_dataset}
folder = "../../dataset/to_sql"

save_dataframes(dfs, folder)


✅ File was save: ../../dataset/to_sql\final_dataset.csv
✅ File was save: ../../dataset/to_sql\final_categories.csv
✅ File was save: ../../dataset/to_sql\final_authors.csv
✅ File was save: ../../dataset/to_sql\mn_author.csv
✅ File was save: ../../dataset/to_sql\mn_categories.csv
✅ File was save: ../../dataset/to_sql\final_lang.csv


In [16]:
def move_files(file_names, source_folder, destination_folder):
    for file_name in file_names:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)

        try:
            with open(source_path, "rb") as src, open(destination_path, "wb") as dst:
                dst.write(src.read())
            print(f"✅ {file_name} was saved successfully.")
        except FileNotFoundError:
            print(f"❌ File {file_name} not found in {source_folder}.")
        except Exception as e:
            print(f"❌ Error processing {file_name}: {e}")

In [17]:
dt_without_trans = ['formats_cleaned.csv','addition_cleaned.csv']
source_folder = "../../dataset/cleaned"
final_path = "../../dataset/to_sql"
move_files(dt_without_trans,source_folder,final_path)

✅ formats_cleaned.csv was saved successfully.
✅ addition_cleaned.csv was saved successfully.
